In [13]:
import csv
from joblib import load
import os
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
from joblib import dump
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [14]:
xTestData = pd.read_csv("test.csv")
xTestData = xTestData.reset_index().set_index('id').drop('index', axis=1)

selectionOfFeatures = ['area', 'missingm5', 'missingm3', 'measurement_17', 'measurement_2', 'measurement_1', 'measurement_0', 'loading']

xTestData['missingm3'] = xTestData.measurement_3.isna()
xTestData['missingm5'] = xTestData.measurement_5.isna()

testFeatures = []
for values in xTestData.columns:
    if values == 'loading' or (values.startswith('measurement') and values != 'measurement_17'):
        testFeatures.append(values)
chosenImputer = KNNImputer(missing_values=np.nan, n_neighbors=15)
chosenImputer.fit(xTestData[testFeatures])
xTestData[testFeatures] = chosenImputer.transform(xTestData[testFeatures])

###
# float_features = []
# for feature in xTestData.columns:
#     if xTestData[feature].dtypes == 'float64':
#         float_features.append(feature)

# standardAverage = []
# for feature in float_features:
#     if feature != 'loading':
#         standardAverage.append(feature)
        
# xTestData['measurement3to17Average'] = xTestData[standardAverage].mean(axis=1)
# xTestData['measurement3to17stdev'] = xTestData[standardAverage].std(axis=1)
###

comparableColumns = ['measurement_3', 'measurement_4', 'measurement_5','measurement_6', 'measurement_7', 'measurement_8', 'measurement_9']
xTestData['measurement17predictions'] = ""
test_list = ['F', 'G', 'H', 'I']
for result in test_list:
    train_source = xTestData[(xTestData['product_code'] == result) & ~pd.isnull(xTestData.measurement_17)]
    LRofMeasurement17 = LinearRegression().fit(train_source[comparableColumns], train_source['measurement_17'])
    sourceForPrediction = xTestData[xTestData['product_code'] == result]
    xTestData.loc[sourceForPrediction.index, 'measurement17predictions'] = LRofMeasurement17.predict(sourceForPrediction[comparableColumns])
    xTestData.loc[sourceForPrediction[pd.isnull(sourceForPrediction.measurement_17)].index, 'measurement_17'] = xTestData.loc[sourceForPrediction[pd.isnull(sourceForPrediction.measurement_17)].index, 'measurement17predictions']
    
PCAObjectives = [
    'measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
    'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12',
    'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'
]

pcaApplied = PCA(n_components=1)
xTestData['pcaApplied'] = pcaApplied.fit_transform(xTestData[PCAObjectives])

xTestData['measurement_2'] = xTestData['measurement_2'].clip(11, None)

xTestData['area'] = xTestData['attribute_2'] * xTestData['attribute_3']

dropped = ['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
            'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12',
            'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16']
xTestData = xTestData.drop(dropped, axis=1)
dropped2 = ['product_code', 'attribute_3', 'attribute_2', 'attribute_1', 'attribute_0']
xTestData = xTestData.drop(dropped2, axis=1)
xTestData[selectionOfFeatures] = StandardScaler().fit_transform(xTestData[selectionOfFeatures])

In [15]:
model = LogisticRegression(penalty='l1', C=0.01, solver='liblinear', random_state=1)
model = load('predictions.joblib') 
y_pred = model.predict_proba(xTestData)[:,1]

In [16]:
sample = pd.read_csv('sample_submission.csv', index_col='id')
sample['id'] = xTestData.index
sample['failure'] =  y_pred
sample.to_csv("109550203.csv",index=False)